
# Text Generation with Neural Networks

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

## Functions for Processing Text

### Reading in files as a string text

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [229]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read().lower()
    return str_text

In [264]:
text = read_file('/content/drive/MyDrive/hiparis/moby_dick_four_chapters.txt')
text = text.replace("\n", " ")

In [265]:
text[:600]

'call me ishmael.  some years ago--never mind how long precisely--having little or no money in my purse, and nothing particular to interest me on shore, i thought i would sail about a little and see the watery part of the world.  it is a way i have of driving off the spleen and regulating the circulation.  whenever i find myself growing grim about the mouth; whenever it is a damp, drizzly november in my soul; whenever i find myself involuntarily pausing before coffin warehouses, and bringing up the rear of every funeral i meet; and especially whenever my hypos get such an upper hand of me, that'

### Tokenize and Clean Text

In [266]:
# The unique characters in the 4th chapter of moby dick, the vocabulary of this character based text generation model
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

40 unique characters


In [267]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [268]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [269]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [286]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
seq_length = 10
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor([b'c' b'a' b'l' b'l' b' ' b'm' b'e' b' ' b'i' b's' b'h'], shape=(11,), dtype=string)


In [287]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'call me ish'
b'mael.  some'
b' years ago-'
b'-never mind'
b' how long p'


In [288]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [305]:
dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

print(dataset)

Input : b'call me is'
Target: b'all me ish'
<_MapDataset element_spec=(TensorSpec(shape=(10,), dtype=tf.int64, name=None), TensorSpec(shape=(10,), dtype=tf.int64, name=None))>


#### Training batches

In [290]:
BATCH_SIZE = 32
BUFFER_SIZE = 1000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(32, 10), dtype=tf.int64, name=None), TensorSpec(shape=(32, 10), dtype=tf.int64, name=None))>

# Creating an LSTM based model

In [291]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 512

In [292]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(rnn_units, return_sequences=True)
    self.lstm1 = tf.keras.layers.LSTM(rnn_units, return_sequences=True)
    self.lstm2 = tf.keras.layers.LSTM(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.lstm.get_initial_state(x)
    x = self.lstm(x, initial_state=states, training=training)
    x = self.lstm1(x, training=training)
    x, mem_states, car_states = self.lstm2(x, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, mem_states, car_states
    else:
      return x

In [293]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

model.summary()

(32, 10, 41) # (batch_size, sequence_length, vocab_size)
Model: "my_model_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_27 (Embedding)    multiple                  10496     
                                                                 
 lstm_49 (LSTM)              multiple                  1574912   
                                                                 
 lstm_50 (LSTM)              multiple                  2099200   
                                                                 
 lstm_51 (LSTM)              multiple                  2099200   
                                                                 
 dense_27 (Dense)            multiple                  21033     
                                                                 
Total params: 5,804,841
Trainable params: 5,804,841
Non-trainable params: 0
______________________________________________________

### Training the Model

In [294]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam', loss=loss)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback], verbose=2)

Epoch 1/30
175/175 - 7s - loss: 2.8800 - 7s/epoch - 38ms/step
Epoch 2/30
175/175 - 1s - loss: 2.3910 - 1s/epoch - 7ms/step
Epoch 3/30
175/175 - 1s - loss: 2.1721 - 1s/epoch - 8ms/step
Epoch 4/30
175/175 - 2s - loss: 2.0250 - 2s/epoch - 9ms/step
Epoch 5/30
175/175 - 1s - loss: 1.9065 - 1s/epoch - 7ms/step
Epoch 6/30
175/175 - 1s - loss: 1.8144 - 1s/epoch - 7ms/step
Epoch 7/30
175/175 - 1s - loss: 1.7275 - 1s/epoch - 7ms/step
Epoch 8/30
175/175 - 1s - loss: 1.6444 - 1s/epoch - 7ms/step
Epoch 9/30
175/175 - 1s - loss: 1.5702 - 1s/epoch - 7ms/step
Epoch 10/30
175/175 - 1s - loss: 1.4911 - 1s/epoch - 7ms/step
Epoch 11/30
175/175 - 1s - loss: 1.4104 - 1s/epoch - 7ms/step
Epoch 12/30
175/175 - 1s - loss: 1.3313 - 1s/epoch - 7ms/step
Epoch 13/30
175/175 - 2s - loss: 1.2462 - 2s/epoch - 9ms/step
Epoch 14/30
175/175 - 1s - loss: 1.1654 - 1s/epoch - 7ms/step
Epoch 15/30
175/175 - 1s - loss: 1.0903 - 1s/epoch - 7ms/step
Epoch 16/30
175/175 - 1s - loss: 1.0159 - 1s/epoch - 7ms/step
Epoch 17/30
175/

# Generating New Text

In [295]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=0.6):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    predicted_logits, mem_states, carry_states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    predicted_logits = predicted_logits + self.prediction_mask

    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    predicted_chars = self.chars_from_ids(predicted_ids)

    return predicted_chars, states

In [296]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [324]:
start = time.time()
states = None
next_char = tf.constant(['The sea is full of'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

The sea is full of be olind we d be soo the thaind t tou t t haroolat tht blind ind the heres hanthes osu t goule thin 

________________________________________________________________________________

Run time: 0.2990429401397705


Results are not really good, that's probably due to the fact that we don't have much data. Moreover, a character-based text generation RNN model needs more data to be accurate

# Try to build a model unsig pre trained GPT
---



In [299]:
!pip install -q keras-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 105.8 MB/s eta 0:00:00


In [300]:
import keras_nlp
from tensorflow import keras

In [301]:
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

497986112/497986112 [==============================] - 14s 0us/step


In [303]:
start = time.time()

output = gpt2_lm.generate("My interview at Institut Polytechnique de Paris went", max_length=50)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
My interview at Institut Polytechnique de Paris went well and I was able to get a better understanding of the project, which will help me better understand how this project works. I am happy to say that the project will be a good fit
TOTAL TIME ELAPSED: 16.34s


In [338]:
text = read_file('/content/drive/MyDrive/hiparis/moby_dick_four_chapters.txt')
text = text.replace("\n", " ")
text = text.split(".")
print(text[0])

call me ishmael


In [339]:
num_epochs = 10

train_ds = (
    tf.data.Dataset.from_tensor_slices(text)
    .batch(64)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-4,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)

loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

gpt2_lm.summary()

gpt2_lm.fit(train_ds, epochs=num_epochs, verbose = 2)

Preprocessor: "gpt2_causal_lm_preprocessor"
__________________________________________________________________________________________________
 Tokenizer (type)                                    Vocab #     
 gpt2_tokenizer (GPT2Tokenizer)                      50257       
__________________________________________________________________________________________________
                                                                                                  
Model: "gpt2_causal_lm"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 padding_mask (InputLayer)      [(None, None)]       0           []                               
                                                                                                  
 token_ids (InputLayer)         [(None, None)]       0           []                               
        

In [341]:
output = gpt2_lm.generate("The sea", max_length=200)
print(output)

The sea-gods had ordained that he should soon become my shipmate (though i cannot tell why it was exactly that those stage managers, the fates, put me down for this shabby part of a whaling voyage, when others were set down for magnificent parts in high tragedies, and short and easy parts in genteel comedies, and jolly parts in farces--though i cannot tell why this was exactly; yet, now that i recall all the circumstances, i think i can see a little into the springs and motives which being cunningly presented to me under various disguises, induced me to set about performing the part i did, besides cajoling me into the


The results are really good and the text is significantly influenced by the chapter 4 of Moby dick used to fine tune GPT. 

## Further improvements

A word based model could be implemented to be compared with the character-based model. 
Transfer Learning seems to be anyway the best method to achieve good results in NLP tasks